In [1]:
from tqdm import trange
import numpy as np


from Game.BlackJack import BlackJack
from Agent.random.randomagent import RandomAgent
from Agent.nn.nnagent import nnAgent
from Agent.LLM.LLM_Agent import LLMAgent
from Agent.base.baseAgent import BaseAgent
from Agent.qlearning.qLearningAgent import qLearningAgent

In [2]:
# agent = nnAgent()
game = BlackJack("novel")
agent = qLearningAgent()

In [3]:
results = {"win": 0, "lose": 0, "draw": 0}
rounds = 100000
for i in trange(rounds):
    game.start()
    # print("Dealer shows:", game.format_cards(game.dealer_hand[:1]))
    while game.status == "continue":
        action = agent.choose_action(game)
        game.play(action, "basic")
        if action == "stay":
            break
    # print(f"dealer's hand: {game.format_cards(game.dealer_hand)} {game.get_dealervalue()}  player's hand: {game.format_cards(game.player_hand)} {game.get_playervalue()}")
    results[game.game_result()] += 1
    
    if len(game.deck) < 20:
        game.reset()
        
print(f"win_rate: {results['win']/rounds}, lose_rate: {results['lose']/rounds}, draw_rate: {results['draw']/rounds}")

100%|██████████| 100000/100000 [00:00<00:00, 118361.46it/s]

win_rate: 0.52304, lose_rate: 0.39973, draw_rate: 0.07723
